<a href="https://colab.research.google.com/github/JCaballerot/Statistics-DataScience/blob/main/03_Pruebas_de_Hip%C3%B3tesis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://storage.googleapis.com/kaggle-datasets-images/1660669/2724653/194571a4f49254b35ba1c8fd6b6d6e07/dataset-cover.jpg?t=2021-10-20-14-39-34" alt="HTML5 Icon" style="width: 600px; height: 450px;">
<div style="text-align: center"></div>

# **Laboratorio de Análisis de Pruebas A/B y Pruebas de Hipótesis**


## **Índice**

1. [Introducción](#1)
2. [Descripción del Conjunto de Datos](#2)
3. [Análisis Exploratorio de Datos](#3)
4. [Pruebas de Hipótesis y Aplicación](#4)
   - 4.1. Prueba de Hipótesis para una Muestra
   - 4.2. Prueba de Hipótesis para Dos Muestras Independientes
   - 4.3. Prueba de Hipótesis para Proporciones
5. [Conclusiones](#5)
6. [Desafío: Ejercicios de Práctica](#6)
7. [Referencias](#7)

---

---


<a name="1"></a>
## **1. Introducción**

El objetivo de este laboratorio es aplicar conceptos de pruebas de hipótesis en el contexto de una prueba A/B utilizando un conjunto de datos de marketing de Kaggle. En este análisis, buscamos entender si existe una diferencia significativa en la tasa de conversión entre dos grupos: aquellos expuestos a un anuncio de producto (`ad`) y aquellos que solo vieron un anuncio de servicio público (`psa`).




<a name="2"></a>
## **2. Descripción del Conjunto de Datos**

El conjunto de datos proviene de [Kaggle: Marketing A/B Testing](https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing). Contiene datos de una prueba A/B que evaluó la efectividad de una estrategia de marketing.

**Variables principales:**

- `index`: Índice de la fila.
- `user id`: Identificador único de cada usuario.
- `test group`: Grupo al que pertenece el usuario. (`ad` para los que vieron un anuncio de producto y `psa` para los que vieron solo un anuncio de servicio público).
- `converted`: Indica si el usuario compró el producto (`True`) o no (`False`).
- `total ads`: Cantidad de anuncios que vio cada persona.
- `most ads day`: Día en que la persona vio la mayor cantidad de anuncios.
- `most ads hour`: Hora del día en que la persona vio la mayor cantidad de anuncios.


<a name="3"></a>
## **3. Análisis Exploratorio de Datos**

### **3.1. Importación de Librerías y Datos**


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Cargar el conjunto de datos
df = pd.read_csv('marketing_AB_testing.csv')

### **3.2. Descripción de las Variables**


Primero, observamos las primeras filas del dataset para entender mejor la estructura de los datos:

In [ ]:
df.head()


## **3.3. Estadística Descriptivas**


Distribución de Usuarios por Grupo


In [ ]:
# Distribución de Usuarios por Grupo
print("Distribución de Usuarios por Grupo:")
print(df['test group'].value_counts())


Tasa de Conversión por Grupo


In [ ]:
# Tasa de Conversión por Grupo
print("\nTasa de Conversión por Grupo:")
conversion_rates = df.groupby('test group')['converted'].mean()
print(conversion_rates)


Análisis de Anuncios Totales y Hora de Máxima Exposición


In [ ]:
# Análisis de Anuncios Totales y Hora de Máxima Exposición por Grupo
print("\nAnálisis de Anuncios Totales y Hora de Máxima Exposición por Grupo:")
ads_stats = df.groupby('test group')[['total ads', 'most ads hour']].mean()
print(ads_stats)


Este análisis nos permite observar las tasas de conversión en cada grupo y explorar las variables total ads y most ads hour para ver si existen diferencias iniciales en la interacción de los usuarios entre los dos grupos.

<a name="4"></a>
## **4. Pruebas de Hipótesis y Aplicación**

A continuación, se describen y aplican las pruebas de hipótesis para evaluar diferencias en la tasa de conversión entre el grupo ad y el grupo psa.

Aplicaremos pruebas de hipótesis para determinar si las diferencias observadas son estadísticamente significativas.


###**4.1. Prueba de Hipótesis para una Muestra**

**Pregunta:** ¿La tasa de conversión promedio en el grupo `psa` es igual al 5% esperado por campañas anteriores?

**Hipótesis:**

- **\(H_0\):** La tasa de conversión en el grupo `psa` es igual al 5% (\( $\mu$
 = 0.05 \)).
- **\(H_a\):** La tasa de conversión en el grupo `psa` es diferente al 5% (\( $\mu$
$\neq$0.05\)).


**Aplicación de la Prueba t para una Muestra (Grupo psa)**


In [ ]:
# Filtrar el grupo `psa`
psa_group = df[df['test group'] == 'psa']

# Datos del grupo `psa`
n_psa = len(psa_group)
mean_psa = psa_group['converted'].mean()
std_psa = psa_group['converted'].std(ddof=1)

# Parámetros de la hipótesis
mu_0 = 0.05  # Tasa de conversión esperada
alpha = 0.05

# Estadístico t
t_stat = (mean_psa - mu_0) / (std_psa / np.sqrt(n_psa))

# Grados de libertad y p-valor
df_psa = n_psa - 1
p_value = stats.t.sf(np.abs(t_stat), df=df_psa) * 2  # Prueba bilateral

print(f"\nPrueba de Hipótesis para una Muestra (Grupo `psa`):")
print(f"Estadístico t: {t_stat:.4f}")
print(f"p-valor: {p_value:.4f}")


Interpretación: Si el p-valor es menor que 0.05, rechazamos 𝐻0​
 , lo que indicaría una diferencia significativa entre la tasa de conversión del grupo psa y la tasa histórica del 5%.

### **4.2. Prueba de Hipótesis para Dos Muestras Independientes**


**Pregunta:** ¿Existe una diferencia significativa en la tasa de conversión entre los grupos `ad` y `psa`?

**Hipótesis:**

- **\( H_0 \):** Las tasas de conversión son iguales en ambos grupos $(\mu_{\text{ad}} = \mu_{\text{psa}}).$
- **\( H_a \):** Las tasas de conversión son diferentes $(\mu_{\text{ad}} \neq \mu_{\text{psa}}).$


**Aplicación de la Prueba t para Dos Muestras Independientes**


In [ ]:
# Filtrar el grupo `ad`
ad_group = df[df['test group'] == 'ad']

# Datos del grupo `ad`
n_ad = len(ad_group)
mean_ad = ad_group['converted'].mean()
std_ad = ad_group['converted'].std(ddof=1)

# Prueba t para muestras independientes (Welch's t-test si las varianzas no son iguales)
t_stat_ind, p_value_ind = stats.ttest_ind(
    psa_group['converted'],
    ad_group['converted'],
    equal_var=False  # Welch's t-test
)

print(f"\nPrueba de Hipótesis para Dos Muestras (Grupo `ad` vs Grupo `psa`):")
print(f"Estadístico t: {t_stat_ind:.4f}")
print(f"p-valor: {p_value_ind:.4f}")


Interpretación: Si el p-valor es menor que 0.05, rechazamos 𝐻0, lo que sugiere una diferencia significativa en la tasa de conversión entre los dos grupos.

### **4.3. Prueba de Hipótesis para Proporciones**


**Pregunta:** ¿Existe una diferencia significativa en la proporción de usuarios que convirtieron entre el grupo ad y el grupo psa?


**Hipótesis:**

- **\( H_0 \):** Las proporciones de conversión son iguales $(p_{\text{ad}} = p_{\text{psa}}).$
- **\( H_a \):** Las proporciones de conversión son diferentes $(p_{\text{ad}} \neq p_{\text{psa}}).$


**Aplicación de la Prueba Z para Proporciones**


In [ ]:
# Contadores de conversiones
success_psa = psa_group['converted'].sum()
success_ad = ad_group['converted'].sum()

# Tamaños de las muestras
p_psa = success_psa / n_psa
p_ad = success_ad / n_ad
p_pool = (success_psa + success_ad) / (n_psa + n_ad)

# Estadístico Z para proporciones
Z_stat = (p_psa - p_ad) / np.sqrt(p_pool * (1 - p_pool) * (1/n_psa + 1/n_ad))
p_value_prop = stats.norm.sf(np.abs(Z_stat)) * 2  # Prueba bilateral

print(f"\nPrueba de Hipótesis para Proporciones (Conversión Grupo `ad` vs Grupo `psa`):")
print(f"Estadístico Z: {Z_stat:.4f}")
print(f"p-valor: {p


Interpretación: Si el p-valor es menor que 0.05, rechazamos 𝐻0, indicando una diferencia significativa en las proporciones de conversión entre ambos grupos.

---

<a name="5"></a>
## **5. Conclusiones**

A partir de los resultados de las pruebas de hipótesis, determinaremos si las diferencias observadas en las tasas de conversión son estadísticamente significativas y si la exposición al anuncio de producto (`ad`) tiene un impacto en comparación con el anuncio de servicio público (`psa`).

---

<a name="6"></a>
## **6. Desafío: Ejercicios de Práctica**

Para afianzar lo aprendido, se proponen los siguientes ejercicios:

1. **Prueba de Hipótesis para una Muestra:**
   - Utiliza el grupo `ad` para verificar si su tasa de conversión es mayor al 7%.

2. **Prueba de Hipótesis para Dos Muestras (Total de Anuncios Vistos):**
   - Compara el número promedio de anuncios vistos entre los grupos `ad` y `psa`. ¿Es la diferencia significativa?

3. **Prueba de Hipótesis para Proporciones (Conversión en Horas Pico):**
   - Analiza si la proporción de conversiones durante la hora de mayor exposición es diferente entre los grupos.

4. **Visualización de Datos:**
   - Crea gráficos que representen las tasas de conversión y otras variables clave para visualizar las diferencias.

---


<a name="7"></a>
## **7. Referencias**

- Montgomery, D. C., & Runger, G. C. (2014). **Applied Statistics and Probability for Engineers**. Wiley.
- Rice, J. A. (2006). **Mathematical Statistics and Data Analysis**. Cengage Learning.
- Kaggle Dataset: [Marketing A/B Testing](https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing)

---